In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-06-20"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDatasetV2 import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_Pontos_H,DP_Pontos_H,CV_Pontos_H,CG_H_Last_Game,Media_Odds_Geral_H,DP_Odds_Geral_H,CV_Odds_Geral_H,Saldo_Gols_H,Avg_CG_A,DP_CG_A,CV_CG_A,Avg_Pontos_A,DP_Pontos_A,CV_Pontos_A,CG_A_Last_Game,Media_Odds_Geral_A,DP_Odds_Geral_A,CV_Odds_Geral_A,Saldo_Gols_A,Goals_Last_H,Goals_Last_A,Last_Odd_H,Last_Odd_A,MediaCustoGolHome,MediaCustoGolAway,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
762,2024-06-20,Canadá Cebl,22:00,Edmonton Stingers,Ottawa Blackjacks,1.54,2.37,177.5,1.82,1.88,0.0,0.00,0.00,rNtbJ1MJ,0.649351,0.421941,0.549451,0.531915,0.071292,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,97,90,1.87,2.37,0.000,0.000,0.300204,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
763,2024-06-20,Canadá Cebl,22:30,Saskatchewan Rattlers,Calgary Surge,1.75,1.98,176.5,1.82,1.88,0.0,0.00,0.00,MDu2ILyQ,0.571429,0.505051,0.549451,0.531915,0.076479,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,85,88,2.32,1.64,0.000,0.000,0.087204,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
764,2024-06-20,Colômbia Lbp,21:00,Piratas de Bogota,Caribbean Storm Islands,3.08,1.34,162.5,1.81,1.89,0.0,0.00,0.00,hrq7WoXF,0.324675,0.746269,0.552486,0.529101,0.070944,252.656,54.445256,0.215492,1.2,1.643168,1.369306,318.40,2.996,0.578731,0.193168,-19.0,129.290,7.696714,0.059531,1.8,1.643168,0.912871,117.12,1.364,0.173868,0.127469,37.0,80,96,3.98,1.22,267.936,186.668,0.556727,0.030578,NaN,-0.20,-0.040,-52.000000,0.000000,0.0,0.000000,6.28,1.256,0.270701,0.000000,0.0,0.000000
765,2024-06-20,Porto Rico Bsn,21:00,Capitanes De Arecibo,Gigantes de Carolina,1.47,2.56,187.5,1.85,1.85,0.0,0.00,0.00,EPP09ZHJ,0.680272,0.390625,0.540541,0.540541,0.070897,136.244,11.024513,0.080917,1.2,1.643168,1.369306,128.02,1.566,0.243988,0.155803,-41.0,189.148,39.796458,0.210399,2.4,1.341641,0.559017,139.65,1.866,0.418844,0.224461,39.0,74,95,1.73,1.47,141.492,214.720,0.382504,0.000000,NaN,0.32,0.064,7.343750,0.745923,0.8,0.054077,1.81,0.362,4.309392,0.470193,0.5,0.029807
766,2024-06-20,Mundo Amistoso Internacional,13:00,Alemanha U20,Turquia U20,2.72,1.42,136.5,1.82,1.88,0.0,0.00,0.00,82NTfRH7,0.367647,0.704225,0.549451,0.531915,0.071872,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,66,77,1.49,1.06,0.000,0.000,0.444077,0.022933,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
767,2024-06-20,Mundo Amistosos Internacionais,13:00,Republica Checa F,Finlândia F,1.18,4.42,143.5,1.85,1.85,0.0,0.00,0.00,ANTZOKoJ,0.847458,0.226244,0.540541,0.540541,0.073702,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,0.000,0.000000,0.000000,0.0,0.000000,0.000000,0.00,0.000,0.000000,0.000000,0.0,84,72,1.22,3.98,0.000,0.000,0.818224,0.000000,NaN,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
768,2024-06-20,Venezuela Superliga,20:00,Marinos,Centauros,1.34,3.08,154.5,1.86,1.84,0.0,0.00,0.00,2midrJ5F,0.746269,0.324675,0.537634,0.543478,0.070944,108.172,9.080180,0.083942,1.8,1.643168,0.912871,101.48,1.404,0.194371,0.138441,8.0,170.222,36.414934,0.213926,0.0,0.000000,NaN,159.10,2.358,0.539602,0.228839,-54.0,86,74,1.18,2.15,104.862,165.238,0.556727,0.007644,NaN,0.03,0.006,56.666667,0.000000,0.0,0.000000,-5.00,-1.000,-2.080000,0.000000,0.0,0.000000
769,2024-06-20,Mundo Amistoso Internacional,10:30,Republic

## Apostas do dia

### Back Home V3 (QCut)

In [9]:
# Duplicando o dataset
df_bh_v3 = jogos.copy()

df_bh_v3 = df_bh_v3[(df_bh_v3['P(H)'] > 0.35) & (df_bh_v3['P(H)'] < 0.65)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_H_Last_Game'] > 0) & (df_bh_v3['CG_H_Last_Game'] <= 170)]
df_bh_v3 = df_bh_v3[(df_bh_v3['CG_A_Last_Game'] > 160)]

df_bh_v3.replace([np.inf, -np.inf], 0, inplace=True)

if not df_bh_v3.empty:
    df_bh_v3.to_csv(f'apostas_do_dia/{data_var}/back_home_v3_{data_var}.csv', index=False)

    print('------------ MÉTODO BACK HOME V3 (QCUT) ------------')
    print()

    df_bh_v3['Bet'] = 'Back Home'
    ligs = df_bh_v3['League'].unique().tolist()

    display(df_bh_v3[['Time', 'League', 'Home', 'Away', 'Odds_H', 'Bet']])
    print()
else: print('Sem jogos')

------------ MÉTODO BACK HOME V3 (QCUT) ------------



,Time,League,Home,Away,Odds_H,Bet
770,20:00,Venezuela Superliga,Guaiqueries,Heroes de Falcon,1.57,Back Home
778,19:30,Brasil Lbf Feminina,Santo André F,Campinas F,1.65,Back Home
781,21:00,Porto Rico Bsn,Piratas de Quebradillas,Aguada Santeros,1.69,Back Home
